In [1]:
import numpy as np
import pandas as pd
import voila
show_tracebacks=True

import warnings
warnings.filterwarnings("ignore")

In [2]:
from IPython.core.display import display, HTML
display(HTML('<h1><center>Some Basic Visualisation for PRAIS Users!</center></h1>'))

In [4]:
#views a sample dataset found online
users = pd.read_csv("C:\\Users\\boconnor\\OneDrive - UNCCD\\GIS\\Practice_Python\\Voila_Jupyter\\Sample_Dataset\\user.csv", index_col=0)
users.head()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xef in position 10: unexpected end of data

In [ ]:
from IPython.core.display import display, HTML
display(HTML('<h3><center>This is a map of the number of users per country!</center></h1>'))

In [ ]:
#Make map of no. of users per country using Choropleth Maps with go.Choropleth
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go

df = users['country'].replace("US", "United States").value_counts()

iplot([go.Choropleth(
    locationmode='country names',
    locations=df.index.values,
    text=df.index,
    z=df.values,
    autocolorscale = True,
    colorbar_title = "Number of users per country",
    colorscale = 'YlOrRd')])


In [ ]:
from IPython.core.display import display, HTML
display(HTML('<h3><center>Metric 1: Time accumulated across users per country since PRAIS 4 launched</center></h1>'))

In [ ]:
#Convert times from string to timestamp
users["last_seen"] = pd.to_datetime(users["last_seen"])
users["date_joined"] = pd.to_datetime(users["date_joined"])
# Calculate difference between date joined and last seen to compute time spent on PRAIS 4
users['Time_Spent_PRAIS4'] = users['last_seen'] - users['date_joined']

#Convert new calculated field from string to timestamp but only extract the days (ignore h,m,s part)
users['Time_Spent_PRAIS4'] = pd.to_timedelta(users.Time_Spent_PRAIS4,errors='coerce').dt.days


# Create new data frame to aggregate by country and sum time spent and store result
df1 = users.groupby('country', as_index=False)['Time_Spent_PRAIS4'].sum()
#print (df1)

#Create the column chart
data = [go.Bar(
   x = df1['country'],
   y =  df1['Time_Spent_PRAIS4']
)]

#Define layout elements
layout = go.Layout({ "title":"Time span between launch of PRAIS 4 and last use per country"})

fig = go.Figure(data=data, layout = layout)
fig.update_yaxes(
        title_text = "No. of days for all users per country",
        title_standoff = 25)

# Set figure title
fig.update_layout(title_text="Time span between launch of PRAIS 4 and last use per country")

#Sort to be in decreasing order
fig.update_layout(xaxis={'categoryorder':'total descending'})


iplot(fig)

In [ ]:
from IPython.core.display import display, HTML
display(HTML('<h3><center>Metric 2: Number of active users per country</center></h1>'))

In [ ]:
# No. of users with account per country
df2 = users.assign(n=0).groupby('country')['n'].count().reset_index()

# No. of users with account per country who never logged in
df3 = users.last_login.isnull().groupby([users['country']]).sum().astype(int).reset_index(name='users never logged in')


#Create the column chart
#data = [go.Bar(
#   x = df2['country'],
#   y =  df2['n']
#)]

#fig = go.Figure(data=data)

# Set figure title
#fig.update_layout(title_text="Number of users of PRAIS 4 per country")



#Sort to be in decreasing order
#fig.update_layout(xaxis={'categoryorder':'total descending'})

#iplot(fig)

In [ ]:
# Create stacked column figure
fig = go.Figure(data=[
    go.Bar(name='Active Users', x=df2['country'], y=df2['n']),
    go.Bar(name='Non Active Users', x=df3['country'], y=df3['users never logged in'])
])
# Change the bar mode
fig.update_layout(barmode='stack')
#Sort to be in decreasing order
fig.update_layout(xaxis={'categoryorder':'total descending'})
# Set y-axis title
fig.update_yaxes(
        title_text = "Total No. of users",
        title_standoff = 25)

# Set figure title
fig.update_layout(title_text="The number of users who never logged into PRAIS 4 as a proportion of all users per country")

iplot(fig)